In [2]:
import cv2
import numpy as np
from skimage.util import random_noise
 
# Load the image
img = cv2.imread("pexels-michael-block-1691617-3225517.jpg")
 
# Add salt-and-pepper noise to the image.
noise_img = random_noise(img, mode='s&p',amount=0.3)
 
# The above function returns a floating-point image
# on the range [0, 1], thus we changed it to 'uint8'
# and from [0,255]
noise_img = np.array(255*noise_img, dtype = 'uint8')
 
# Display the noise image
cv2.imshow('blur',noise_img)
cv2.waitKey(0)

-1

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from PIL import Image

beta1 = 0.0001  
beta2 = 0.02 
timesteps = 500

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#b_t = (beta2 - beta1) * torch.linspace(0, 1, timesteps + 1, device=device) + beta1
b_t = torch.full((timesteps + 1,), 0.02, device=device)
a_t = 1 - b_t
ab_t = torch.cumsum(a_t.log(), dim=0).exp().to(device)  # Move ab_t to the same device
ab_t[0] = 1.0  # Ensure the initial value is 1.0 (100% of the original image)

image_path = 'pexels-michael-block-1691617-3225517.jpg'  # Replace with your image path
img = Image.open(image_path)  
img = img.convert('RGB')  
img_array = np.array(img)
img_array = img_array / 255.0  # Normalize to [0, 1]
img_tensor = torch.tensor(img_array, dtype=torch.float32, device=device).permute(2, 0, 1).unsqueeze(0)  # Convert to tensor and add batch dimension, move to device

plt.imshow(np.clip(img_array, 0, 1))  # Ensure values are in [0, 1]
plt.title(f"Original Image")
plt.axis('off')
plt.show()

noise = torch.randn_like(img_tensor, device=device)
encrypted_image = ab_t.sqrt()[timesteps, None, None, None] * img_tensor + (1 - ab_t[timesteps, None, None, None]).sqrt() * noise

encrypted_image = encrypted_image.squeeze().permute(1, 2, 0).cpu().numpy()  # Convert back to HWC format for visualization

plt.imshow(np.clip(encrypted_image, 0, 1))  # Ensure values are in [0, 1]
plt.title(f"Encrypted Image at Timestep {100}")
plt.axis('off')
plt.savefig('encrypted_image.png')  # Save the image
plt.show()

differences = []
x_t = torch.tensor(encrypted_image.copy(), dtype=torch.float32, device=device).permute(2, 0, 1).unsqueeze(0)

for t in range(100, -1, -1):
    x_t_minus_1 = (x_t - ((1 - a_t[t])/torch.sqrt(1-ab_t[t])) * noise) / torch.sqrt(a_t[t])
    x_t = x_t_minus_1

    decrypted_image = x_t.squeeze().permute(1, 2, 0).cpu().numpy()
    '''plt.imshow(np.clip(decrypted_image, 0, 1))  # Ensure values are in [0, 1]
    plt.title(f"Decrypted Image at Timestep {t}")
    plt.axis('off')
    plt.show() '''

    difference = torch.mean((img_tensor - x_t) ** 2).item()  # Mean squared error
    differences.append(difference)

# Plot the differences
plt.figure(figsize=(10, 5))
plt.plot(range(100, -1, -1), differences, label='Difference')
plt.xlabel('Timestep')
plt.ylabel('Difference (MSE)')
plt.title('Difference between img_tensor and x_t over timesteps')
plt.legend()
plt.grid(True)
plt.show()